<a href="https://colab.research.google.com/github/jinseriouspark/pricing/blob/main/%5Bdynamic_pricing%5D_04_multi_armed_bandit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dynamic pricing - 04. Multi Armed Bandit


- 의미 : dynamic pricing 에서는 멀티 암드 밴딧 게임을 사용해서 상품이나 서비스의 가격을 정할때, 어떤 가격이 가장 잘 팔리는지 알아보려고 한다. 여러가지 가격을 시도해보면서, 어떤 가격이 사람들에게 가장 합리적인지 알아내고, 해당 가격을 적용하고자 함


- 방법
  1. 탐험 exploration : 각각의 가격대를 몇번씩 시도하며 유저들의 반응을 살펴봄 (각각의 매출 분포를 이해)
  2. 활용 Exploitation : 어느정도 탐험을 하고난 다음, 유저들의 반응이 가장 높았던 조건의 가격대를 선정하고자 함

- 적용:  여러가지 가격을 시도하여 각 가격에서의 판매량(보상)을 관찰하고, 최적의 가격을 찾아내는 데 사용

##  1. Epsilon-greedy algorithm
- 미래를 생각하지 않고 각 단계에서 가장 최선의 선택을 하는 기법. 각 단계에서 최선의 선택을 한 것이 전체적으로도 최선이라는 것을 의미함

- e 의 확률로 탐색을 하고, 1-e 의 확률로 현재까지 가장 좋은 선택을 활용함

In [39]:
import numpy as np


class EpsilonGreedyBandit:
  def __init__(self, k, epsilon, initial_value =0):
    self.k = k # 선택지의 개수
    self.epsilon = epsilon
    self.q_values = np.full(k, initial_value, dtype=float) # 각 선택의 예상 보상
    self.action_counts = np.zeros(k, dtype = int) # 각 선택지의 선택 횟수

  def select_action(self):
    if np.random.rand() < self.epsilon:
      return np.random.randint(self.k) # 탐색 : epsilon보다 작으면 탐색
    else:
      return np.argmax(self.q_values) # 활용 : 예상 보상이 가장 높은 선택지 채택

  def update(self, chosen_action, reward):
    self.action_counts[chosen_action] += 1 # 선택지 값 업데이트
    n = self.action_counts[chosen_action] # 해당 선택지의 action수
    value = self.q_values[chosen_action] # 해당 시점의 보상

    # 새로운 보상 값 계산
    # chosen_action 의 값의 대부분은 기존 보상을, 1 / n 의 비율로 새로운 reward를 반영
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.q_values[chosen_action] = new_value

In [42]:
# 예제 : 선택가능한 것 5개, 가장 좋은 액션 선택할 확률 (1 - eplion)
k = 5
epsilon = 0.1
bandit = EpsilonGreedyBandit(k, epsilon)

# 시뮬레이션
n_simulations = 1000
true_rewards = [0.1, 0.5, 0.8, 0.4, 0.9] # 실제 각 액션의 보상
rewards = np.zeros(n_simulations)

#
for i in range(n_simulations):
    chosen_action = bandit.select_action()
    reward = np.random.rand() < true_rewards[chosen_action]
    bandit.update(chosen_action, reward)
    rewards[i] = reward

In [37]:
print('각 액션의 예상 보상', bandit.q_values)
print('총 예상 보상', np.sum(rewards))

각 액션의 예상 보상 [0.22727273 0.41666667 0.8046875  0.30769231 0.89393939]
총 예상 보상 824.0


## 2. Upper Confidence Bound, UCB
- 각 액션의 평균 보상과 탐색 요소를 결합하여 신뢰구간이 가장 높은 선택지를 선택함
- 아직 충분히 탐색되지 않은 액션을 더 자주 선택하도록 유도

In [52]:
import numpy as np

class UCBBandit:
  def __init__(self, n_actions):
    self.n_actions = n_actions
    self.action_counts = np.zeros(n_actions) # 각 선택지의 선택 횟수
    self.q_values = np.zeros(n_actions) # 각 선택지의 예상 보상
    self.total_counts = 0 # 전택 선택 횟수

  def select_action(self):
    if 0 in self.action_counts: # 아직 한번도 선택되지 않은 팔이 있다면 우선 선택
      print(self.q_values)
      return np.argmin(self.action_counts)
    else: # UCB값 계산
      # q_values 는 현재까지의 평균 보상
      ucb_values = self.q_values + np.sqrt((2 * np.log(self.total_counts)) / self.action_counts)
      return np.argmax(ucb_values)

  def update(self, chosen_action, reward):
    self.action_counts[chosen_action] += 1
    self.total_counts += 1
    n = self.action_counts[chosen_action]
    value = self.q_values[chosen_action]

    # 새로운 보상 값 계산
    print(n, value, reward, (1 / float(n)) * reward)
    new_value = ((n - 1) / float(n)) * value + (1 / float(n)) * reward
    self.q_values[chosen_action] = new_value


In [53]:
# 예제 사용
n_actions = 5
bandit = UCBBandit(n_actions)

# 시뮬레이션
n_simulations = 1000
true_rewards = [0.1, 0.5, 0.8, 0.4, 0.9]
rewards = np.zeros(n_simulations)
from tqdm import tqdm
for i in tqdm(range(n_simulations)):
    chosen_action = bandit.select_action()
    reward = np.random.rand() < true_rewards[chosen_action]
    bandit.update(chosen_action, reward)
    rewards[i] = reward

print('각 팔의 예상 보상', bandit.q_values)
print('총 예상 보상', np.sum(rewards))

  6%|▌         | 61/1000 [00:00<00:01, 577.91it/s]

[0. 0. 0. 0. 0.]
1.0 0.0 False 0.0
[0. 0. 0. 0. 0.]
1.0 0.0 True 1.0
[0. 1. 0. 0. 0.]
1.0 0.0 True 1.0
[0. 1. 1. 0. 0.]
1.0 0.0 False 0.0
[0. 1. 1. 0. 0.]
1.0 0.0 True 1.0
2.0 1.0 True 0.5
2.0 1.0 True 0.5
2.0 1.0 False 0.0
3.0 1.0 True 0.3333333333333333
3.0 1.0 True 0.3333333333333333
4.0 1.0 False 0.0
4.0 1.0 True 0.25
2.0 0.0 False 0.0
2.0 0.0 False 0.0
5.0 1.0 True 0.2
3.0 0.5 True 0.3333333333333333
6.0 1.0 True 0.16666666666666666
4.0 0.6666666666666666 True 0.25
7.0 1.0 True 0.14285714285714285
5.0 0.75 False 0.0
5.0 0.75 False 0.0
8.0 1.0 False 0.0
3.0 0.0 False 0.0
3.0 0.0 True 0.3333333333333333
4.0 0.3333333333333333 True 0.25
9.0 0.875 True 0.1111111111111111
5.0 0.5 False 0.0
6.0 0.6000000000000001 True 0.16666666666666666
6.0 0.6000000000000001 True 0.16666666666666666
10.0 0.8888888888888888 True 0.1
7.0 0.6666666666666667 False 0.0
7.0 0.6666666666666667 True 0.14285714285714285
11.0 0.8999999999999999 True 0.09090909090909091
8.0 0.7142857142857144 True 0.125
12.0 0.9

 25%|██▌       | 254/1000 [00:00<00:00, 882.10it/s]

 0.6111111111111113 True 0.05263157894736842
20.0 0.6315789473684212 False 0.0
45.0 0.8636363636363634 True 0.022222222222222223
15.0 0.5 False 0.0
46.0 0.8666666666666665 True 0.021739130434782608
47.0 0.8695652173913042 True 0.02127659574468085
48.0 0.8723404255319148 True 0.020833333333333332
49.0 0.8749999999999999 True 0.02040816326530612
50.0 0.8775510204081631 True 0.02
51.0 0.8799999999999999 False 0.0
40.0 0.8205128205128204 True 0.025
41.0 0.8249999999999998 True 0.024390243902439025
42.0 0.8292682926829267 True 0.023809523809523808
43.0 0.8333333333333331 True 0.023255813953488372
44.0 0.8372093023255812 True 0.022727272727272728
45.0 0.8409090909090907 True 0.022222222222222223
46.0 0.8444444444444442 True 0.021739130434782608
47.0 0.8478260869565215 True 0.02127659574468085
48.0 0.8510638297872338 False 0.0
21.0 0.6000000000000002 True 0.047619047619047616
22.0 0.6190476190476193 False 0.0
52.0 0.8627450980392155 False 0.0
49.0 0.833333333333333 True 0.02040816326530612
50

 39%|███▉      | 394/1000 [00:00<00:00, 1051.75it/s]

...

 0.8779527559055118 True 0.00392156862745098
256.0 0.8784313725490196 True 0.00390625
257.0 0.87890625 True 0.0038910505836575876
258.0 0.8793774319066148 True 0.003875968992248062
259.0 0.8798449612403101 True 0.003861003861003861
260.0 0.8803088803088803 True 0.0038461538461538464
261.0 0.8807692307692307 True 0.0038314176245210726
262.0 0.8812260536398467 True 0.003816793893129771
263.0 0.8816793893129771 True 0.0038022813688212928
264.0 0.8821292775665399 True 0.003787878787878788
265.0 0.8825757575757576 True 0.0037735849056603774
266.0 0.8830188679245283 True 0.0037593984962406013
267.0 0.8834586466165414 True 0.003745318352059925
268.0 0.8838951310861424 True 0.0037313432835820895
269.0 0.8843283582089553 True 0.0037174721189591076
270.0 0.8847583643122677 True 0.003703703703703704
271.0 0.8851851851851853 False 0.0
45.0 0.5681818181818181 True 0.022222222222222223
46.0 0.5777777777777777 False 0.0
272.0 0.881918819188192 True 0.003676470588235294
273.0 0.8823529411764708 True 0

 58%|█████▊    | 576/1000 [00:00<00:00, 526.85it/s]

0.889240506329114 True 0.0031545741324921135
318.0 0.889589905362776 True 0.0031446540880503146
319.0 0.889937106918239 True 0.003134796238244514
320.0 0.890282131661442 True 0.003125
321.0 0.890625 False 0.0
47.0 0.5652173913043478 True 0.02127659574468085
48.0 0.5744680851063829 True 0.020833333333333332
49.0 0.5833333333333333 False 0.0
322.0 0.8878504672897196 True 0.003105590062111801
323.0 0.8881987577639752 True 0.0030959752321981426
324.0 0.8885448916408669 False 0.0
114.0 0.7522123893805308 False 0.0
325.0 0.8858024691358025 True 0.003076923076923077
326.0 0.8861538461538462 True 0.003067484662576687
327.0 0.8865030674846626 True 0.0030581039755351682
328.0 0.8868501529051988 True 0.003048780487804878
329.0 0.8871951219512195 True 0.00303951367781155
330.0 0.8875379939209727 True 0.0030303030303030303
331.0 0.8878787878787879 True 0.0030211480362537764
332.0 0.8882175226586103 True 0.0030120481927710845
333.0 0.8885542168674699 True 0.003003003003003003
334.0 0.888888888888889

100%|██████████| 1000/1000 [00:01<00:00, 834.31it/s]

0.0
399.0 0.8793969849246231 True 0.002506265664160401
400.0 0.8796992481203008 True 0.0025
401.0 0.88 True 0.0024937655860349127
402.0 0.8802992518703242 True 0.0024875621890547263
403.0 0.8805970149253731 True 0.0024813895781637717
404.0 0.8808933002481389 True 0.0024752475247524753
405.0 0.8811881188118812 True 0.0024691358024691358
406.0 0.8814814814814815 True 0.0024630541871921183
407.0 0.8817733990147784 True 0.002457002457002457
408.0 0.8820638820638821 False 0.0
54.0 0.5660377358490566 False 0.0
127.0 0.738095238095238 True 0.007874015748031496
128.0 0.7401574803149605 True 0.0078125
129.0 0.7421874999999999 True 0.007751937984496124
130.0 0.7441860465116278 False 0.0
409.0 0.8799019607843137 True 0.0024449877750611247
410.0 0.8801955990220048 True 0.0024390243902439024
411.0 0.8804878048780487 True 0.0024330900243309003
412.0 0.8807785888077858 False 0.0
413.0 0.8786407766990291 True 0.002421307506053269
414.0 0.8789346246973365 False 0.0
131.0 0.7384615384615384 False 0.0
41